This is a testing notebook for loading and analyzing the output of ImageJ's TrackMate plugin (https://imagej.net/TrackMate). The plugin outputs a wealth of data in XML format, and this set of scripts provides an interface for loading and analyzing these data in Python.

In [ ]:
# load the custom module
%load_ext autoreload
%autoreload 2

import os, sys, inspect
import matplotlib.pylab as plt
import numpy as np

# Add source code directory to path to enable module import
curr_frame = inspect.getfile(inspect.currentframe())
curr_dir = os.path.dirname(os.path.abspath(curr_frame))
parent_dir = os.path.dirname(curr_dir)
module_dir = os.path.join(parent_dir, 'src')
os.sys.path.insert(0, module_dir)

import parse_trackmate as pt


In [ ]:
# Load test data

# Path to raw TrackMate data
data_dir = '../data/raw/test_dataset_1'
file_name = 'MMStack_Pos0.ome.test1.xml'
full_file = os.path.join(data_dir, file_name)

# Read data from the provided source file
raw_data = pt.read_trackmate_file(full_file)

In [ ]:
# Parse the data into tracks, creating lists of spots that
# make up each individual track 

raw_tracks = pt.get_raw_tracks(raw_data)
spots = pt.list_spots(raw_data)

processed_tracks = pt.build_spot_lists_by_track(raw_tracks, spots)
track_coords, track_edges, track_spot_ids = processed_tracks

all_track_intensities = pt.pull_property_by_track('MEAN_INTENSITY',
                                               track_spot_ids, spots)

In [ ]:
# Calculate MSD for each particle
from scipy import stats
%matplotlib

t_dsq = list() # time vs sq. displacement for each particle
for track in track_coords:
    # 'track' is an array of x,y,t
    
    track_norm = track - track[0,:] # set x,y,t=0
    xy_sq = np.square(track_norm[:,0:2])
    d_sq = np.sum(xy_sq, axis=1)
    t = track_norm[:,2]
    tds = np.vstack((t, d_sq))
    tds = np.transpose(tds)
    
    t_dsq.append(tds)

# rebuild results as a padded numpy array for calculating means
l_max = max((len(el) for el in t_dsq))

d_sq = np.empty((l_max, len(t_dsq)))
t_total = np.empty((l_max, len(t_dsq)))
for i, x in enumerate(t_dsq):
    len1 = x.shape[0]
    pad_len = l_max - len1
    nanpad = np.empty([pad_len,2])*np.nan
    x_padded = np.concatenate((x,nanpad), axis=0)
    
    d_sq[:,i] = x_padded[:,1]
    t_total[:,i] = x_padded[:,0]
    
mean_t = np.nanmean(t_total, axis=1)
mean_dsq = np.nanmean(d_sq, axis=1)
std_dsq = np.nanstd(d_sq, axis=1)
sterr_dsq = stats.sem(d_sq, axis=1, nan_policy='omit',ddof=0)

# Plot results
f1, axarr = plt.subplots(1,2)
axarr[0].set_title('All tracks')
axarr[0].set_xlabel('Time (s)')
axarr[0].set_ylabel('Position (μm)')
axarr[1].set_title('Mean squared displacements')
axarr[1].set_xlabel('Time (s)')
axarr[1].set_ylabel('MSD (μm^2)')
for x in t_dsq:
    axarr[0].plot(x[:,0], x[:,1])


axarr[1].errorbar(mean_t, mean_dsq, yerr=sterr_dsq, fmt='o')
#axarr[1].plot(mean_t, mean_dsq)

plt.show()

In [ ]:
# Plot some useful stuff


#%matplotlib inline
import matplotlib.pylab as plt
#plt.xkcd()

# Various plotting tools
f2, axarr = plt.subplots(1,2)
axarr[0].set_title('All tracks')
axarr[0].set_xlabel('Position (μm)')
axarr[0].set_ylabel('Position (μm)')
axarr[1].set_xlabel('Time (s)')
axarr[1].set_ylabel('Intensity (A.U.)')
axarr[1].set_title('All intensities')
for i, track in enumerate(track_coords):
    axarr[0].plot(track[:,0], track[:,1])
    axarr[1].plot(track[:,2], all_track_intensities[i])
plt.show()
#plt.plot(track_coords[testind][:,2], all_track_intensities[testind])